In [2]:
import numpy as np
np.set_printoptions(precision = 2)

In [3]:
class RKHS:
    def __init__(self, P):
        self.P = P

        # Checking that matrix P is a square and symmetric matrix
        n,m = P.shape
        if n != m:
            raise Exception("The matrix is not square")
        for i in range(n):
            for j in range(i+1,n):
                if P[i,j] != P[j,i]:
                    raise Exception("Not Symmetric")
            
        # Computing eigenvalues and eigenvectors of P
        eigval, eigvec = np.linalg.eig(P)

        # Checking that all eigenvalues are non-negative implying positive semi-definiteness of P
        for e in eigval:
            if e<0:
                raise Exception("Not Positive semidefinite")
        
        # Computing the Moore-Penrose inverse of P
        D = np.diag(eigval)
        D_plus = np.zeros_like(D)
        for i in range(n):
            if D[i,i] >0:
                D_plus[i,i] = 1/D[i,i] 
        self.P_plus = eigvec @ D_plus @ (eigvec.T)

        # Computing the orthongonal projection onto the column space of P
        D_ = np.zeros_like(D)
        for i in range(n):
            if D[i,i]>0:
                D_[i,i] = 1
        self.Proj = eigvec @ D_ @(eigvec.T) 

    def inner_prod(self, u, v):
        # Checking that u and v are indeed in the column space of P
        tol = 1e-8
        if np.linalg.norm(self.Proj @ u - u) >tol:
            raise Exception("u is not in the column space of P")
        if np.linalg.norm(self.Proj @ v - v) >tol:
            raise Exception("v is not in the column space of P")
        
        # Computing the inner product of u and v in the RKHS induced my matrix P
        innprod = np.dot(self.P_plus@u, v)
        return innprod
    
    def get_orthogonal_basis(self):
        n = self.P.shape[0]
        vecs = []
        # Gram Schmidt orthogonalization process
        for i in range(n):
            u = self.P[:,i]
            u_hat = u.copy()
            for vec in vecs:
                coeff = self.inner_prod(u, vec)
                u_hat = u_hat - coeff * vec
            u_hat_norm_sq = self.inner_prod(u_hat, u_hat)
            if u_hat_norm_sq >0 :
                vecs.append(u_hat/np.sqrt(u_hat_norm_sq))
        return vecs
    
    def reconstruct_P(self): # reconstructing P using the power series expansion of kernel using orthonormal basis 
        orth_basis = self.get_orthogonal_basis()
        n = self.P.shape[0]
        rec_P = np.zeros(shape = (n,n), dtype = float)
        for i in range(n):
            for j in range(n):                
                for vec in orth_basis:
                    rec_P[i,j] += vec[i]*vec[j]
        return rec_P

In [4]:
A = np.array(
    [
        [1, 1, 0],
        [1, 4, 1],
        [0, 1, 1]
    ]
)
rkhs_A = RKHS(A)

In [5]:

u = np.array([0,3,0])
v = np.array([1,0,0])
rkhs_A.inner_prod(u,v)

-1.5000000000000004

In [6]:
# Computing the orthogonal basis of column space of P w.r.t. the norm of the RKHS
rkhs_A.get_orthogonal_basis()

[array([1., 1., 0.]),
 array([0.  , 1.73, 0.58]),
 array([1.36e-16, 5.44e-16, 8.16e-01])]

In [7]:
# Reconstructing P using the above orthonormal basis

rkhs_A.reconstruct_P()

array([[1.00e+00, 1.00e+00, 1.11e-16],
       [1.00e+00, 4.00e+00, 1.00e+00],
       [1.11e-16, 1.00e+00, 1.00e+00]])